# TensorIR: 张量程序抽象案例研究
> 简单了解关于TVM的相关实现

In [1]:
!python3 -m  pip install mlc-ai-nightly -f https://mlc.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://mlc.ai/wheels
     |████████████████████████████████| 43.3 MB 1.3 MB/s 


In [2]:
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
import numpy as np

使用张量程序抽象的主要目的是表示循环和相关的硬件加速选择，如多线程、特殊硬件指令的使用和内存访问。

为了帮助我们更好地解释，我们用下面的张量计算作为示例。

具体地，对于两个大小为 $128 \times 128$ 的矩阵 A 和 B，我们进行如下两步的张量计算。

- $Y_{i, j} = \sum_k A_{i, k} \times B_{k, j}$
- $C_{i, j} = \mathbb{relu}(Y_{i, j}) = \mathbb{max}(Y_{i, j}, 0)$

上面的计算很像在我们神经网络中经常看到的典型的元张量函数：一个线性层与一个 ReLU 激活层。首先，我们使用如下 NumPy 中的数组计算实现这两个操作。


In [3]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

为了说明底层细节，我们将在 NumPy API 的一个受限子集中编写示例 —— 我们称之为 **低级 NumPy**。它使用以下的约定：

- 我们将在必要时使用循环计算。
- 如果可能，我们总是通过 `numpy.empty` 显式地分配数组并传递它们。

下面是其中的一种实现方式👇：

In [4]:
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype="float32")
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)

验证实现是否等价

In [5]:
c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)

## TVMScript的实现
在看过低级 NumPy 示例后，现在我们准备介绍 TensorIR。下面的代码块展示了 `mm_relu` 的 TensorIR 实现。这里的代码是用一种名为 TVMScript 的语言实现的，它是一种嵌入在 Python AST 中的特定领域方言。

In [6]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

Numpy底层实现与TVMScript实现对应：
![](https://mlc.ai/zh/_images/tensor_func_and_numpy.png)


唯一不同的地方在：
```python
with T.block("Y"):
  vi = T.axis.spatial(128, i)
  vj = T.axis.spatial(128, j)
  vk = T.axis.reduce(128, k)
```

这三行包含以下信息：

- 定义了 `vi`、`vj`、`vk` 应被绑定到的位置（在本例中为 `i`、`j` 和 `k`）；
- 声明了 `vi`、`vj`、`vk` 的原始范围（`T.axis.spatial(128, i)` 中的 `128`）；
- 声明了块轴的属性（`spatial`, `reduce`）。

一般情况下为了简单起见，我们会写成：
```python
vi, vj, vk = T.axis.remap("SSR", [i, j, k])
```

In [7]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

## TVM 中实现元张量函数转换
另外一个版本的低级别Numpy的实现：

In [8]:
def lnumpy_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
  Y = np.empty((128, 128), dtype="float32")
  # 将j拆分为j0, j1
  for i in range(128):
    for j0 in range(32):
      for k in range(128):
        for j1 in range(4):
          j = j0 * 4 + j1
          if k == 0:
            Y[i, j] = 0
          Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
  for i in range(128):
    for j in range(128):
      C[i, j] = max(Y[i, j], 0)

c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu_v2(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5) # 验证是否等价

### 等价实现一
> 下面展示在TVMScript如何实现这种转换


工具函数，高亮对应的python代码：

In [9]:
import IPython

def code2html(code):
  """高亮代码"""
  import pygments
  from pygments.lexers import Python3Lexer
  from pygments.formatters import HtmlFormatter
  formatter = HtmlFormatter()
  html = pygments.highlight(code, Python3Lexer(), formatter)
  return "<style>%s</style>%s\n" % (formatter.get_style_defs(".highlight"), html)


IPython.display.HTML(code2html(MyModule.script()))

In [10]:
print(MyModule.script()) # 无高亮

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def mm_relu(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j, k in tir.grid(128, 128, 128):
            with tir.block("Y"):
                vi, vj, vk = tir.axis.remap("SSR", [i, j, k])
                tir.reads(A[vi, vk], B[vk, vj])
                tir.writes(Y[vi, vj])
                with tir.init():
                    Y[vi, vj] = tir.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in tir.grid(128, 128):
            with tir.block("C"):
                vi, vj = tir.axis.remap("SS", [i, j])
                tir.reads(Y[vi, vj])
                tir.writes(C[vi, vj])
                C[vi, vj]

* 建立一个Schedule的类，方便后续操作

In [11]:
sch = tvm.tir.Schedule(MyModule)

* 获取函数的block和对应的循环

In [12]:
block_Y = sch.get_block("Y", func_name="mm_relu")
i, j, k = sch.get_loops(block_Y)  # 就是那个i, j, k 的循环

* 对`j`进行处理, 分别分割成32和4

In [13]:
j0, j1 = sch.split(j, factors=[None, 4])

通过`sch.mod.script()`来查看:

In [14]:
IPython.display.HTML(code2html(sch.mod.script()))

* 要把`j0, k, j1` 顺序改一下

In [15]:
sch.reorder(j0, k, j1)
IPython.display.HTML(code2html(sch.mod.script()))

对比一下之前低级别Numpy实现的程序
```python
def lnumpy_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
  Y = np.empty((128, 128), dtype="float32")
  # 将j拆分为j0, j1
  for i in range(128):
    for j0 in range(32):
      for k in range(128):
        for j1 in range(4):
          j = j0 * 4 + j1
          if k == 0:
            Y[i, j] = 0
          Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
  for i in range(128):
    for j in range(128):
      C[i, j] = max(Y[i, j], 0)
```

### 等价实现二

* 算子融合操作
将`C`的`block`转移到`j0`的`Y`的计算里面

In [18]:
block_c = sch.get_block("C", "mm_relu")
sch.reverse_compute_at(block_c, j0)
IPython.display.HTML(code2html(sch.mod.script()))

* 拆开初始化与求和

In [19]:
block_Y = sch.get_block("Y", "mm_relu")
sch.decompose_reduction(block_Y, k)
IPython.display.HTML(code2html(sch.mod.script()))

上面的`TVMScript`的实现等价于下面的`Numpy`的低级别实现

In [20]:
def lnumpy_mm_relu_v3(A: np.ndarray, B: np.ndarray, C: np.ndarray):
  Y = np.empty((128, 128), dtype='float32')
  for i in range(128):
    for j0 in range(32):
      # 初始化Y_init
      for j1 in range(128):
        j = j0 * 4 + j1
        Y[i, j] = 0
      # 更新Y
      for k in range(128):
        for j1 in range(4):
          j = j0 + 4 * j1
          Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
      # C
      for j1 in range(4):
        j = j0 * 4 + j1
        C[i, j] = max(Y[i, j], 0)

c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu_v2(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5) # 验证是否等价

### 构建与运行`tensor ir` 程序

In [25]:
rt_lib = tvm.build(MyModule, target='llvm') # 创建运行库

将`Numpy`的数组转换为`tvm`对应类型的数组

In [28]:
a_nd = tvm.nd.array(a_np)
b_nd = tvm.nd.array(b_np)
c_nd = tvm.nd.empty((128, 128), dtype="float32")
type(c_nd)

tvm.runtime.ndarray.NDArray

* 拿到对应的函数实现

In [30]:
from numpy.testing import assert_allclose
func_mm_relu = rt_lib["mm_relu"]
func_mm_relu(a_nd, b_nd, c_nd)
             
np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

* **同样，可以使用变换之后的张量函数来跑**

In [31]:
rt_lib_after = tvm.build(sch.mod, target='llvm')
rt_lib_after["mm_relu"](a_nd, b_nd, c_nd)
np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

* 最后我们比较变换前后的运行时间的差别
> 我们跑多轮来进行比较

结果可以看出变换后的函数更快

In [33]:
f_timer_before = rt_lib.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of MyModule % g sec" % f_timer_before(a_nd, b_nd, c_nd).mean)
f_time_after = rt_lib_after.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of transformed sch.mod %g sec" % f_time_after(a_nd, b_nd, c_nd).mean)

Time cost of MyModule  0.00595749 sec
Time cost of transformed sch.mod 0.000895271 sec


下面解释一下为什么变换后更快

In [35]:
IPython.display.HTML(code2html(MyModule.script()))

In [34]:
IPython.display.HTML(code2html(sch.mod.script()))

两者唯一的区别就在于对于`j`的迭代顺序不一样，主要是因为`CPU`读取速度不一样！
![](https://s2.loli.net/2022/07/13/mla9WTO2jJLtR3H.png)

转换后的程序都在附近的位置来访问内存
![](https://s2.loli.net/2022/07/13/UZ86HOzb1ltTsac.png)

本小节主要就是熟悉如何编写`tensor IR`, 与如何变换。后续会继续深入，如何在CPU上得到最佳实现。

## 转换成TensorIR 程序

基于`tensor expression`生成对应的TensorIR程序

In [36]:
from tvm import te
A = te.placeholder((128, 128), "float32", name="A")
B = te.placeholder((128, 128), "float32", name="B")
k = te.reduce_axis((0, 128), "k")
Y= te.compute((128, 128), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="Y")
C = te.compute((128, 128), lambda i, j: te.max(Y[i, j], 0), name="C")

$Y_{i j}=\sum_{k} A_{i k} B_{k j}$

In [37]:
lambda i, j: te.sum(A[i, k] * B[k, j], axis=k)

<function __main__.<lambda>>

In [38]:
te_func = te.create_prim_func([A, B, C]).with_attr({"global_symbol": "mm_relu"})
MyModuleFromTE = tvm.IRModule({"mm_relu": te_func})
IPython.display.HTML(code2html(MyModuleFromTE.script()))

最后还有融合形成TensorIR的

## 总结
传统的开发模式
![](https://s2.loli.net/2022/07/13/cH9faIWnsmpje35.png)

机器学习编译的开发模式（强调**变换**）

![](https://s2.loli.net/2022/07/13/Q5H3rRgvNzwZd2D.png)

未来的课程核心在:
* 用哪些不同的方式表示张量函数(抽象)
* 有哪些更高效的变换